## Refinery Feedstock Optimization ##

*Given the following economic and crude product information, how much of each crude should a refinery process to maximize profitability?*

<img src="refinery-problem.png">

This is a classic problem for teaching Linear Programming/LP modeling! Let's solve it with using Python's scipy.optimize.minimize function, along with some matrix algebra methods from numpy.

**Step 1: Data Entry**

The tables above contain everything we need to solve the problem, but where to begin? Crude cost and operating cost are both constant, so we can start by calculating the total cost for each crude type.

The cost data can be organized into lists, with the index of the list encoding which crude is associated with which price.

In [2]:
given_cost = [110, 110, 115, 120, 120, 110]
given_opcost = [20, 17, 19, 15, 18, 17]

We could add each element of these lists together using a generator...

In [14]:
crude_cost = [given_cost[i]+given_opcost[i] for i in range(len(given_cost))]
print(crude_cost)

[130, 127, 134, 135, 138, 127]


However, in future steps, we will need to use this cost data in matrix multiplication and dot product calculations. That means we need our cost information to be in a numpy array. Let's cast each list to a numpy array and add them together.

First, we need to import numpy.

In [16]:
import numpy as np

Now, we can calculate a `total_cost` array that will hold the total_cost information, and offer more capabilities for future calculation.

In [17]:
crude_cost = np.array(given_cost) + np.array(given_opcost)
print(crude_cost)

[130 127 134 135 138 127]


With our total crude cost ($/BBL) calculated, its time to enter the crude product yield (assay) information. Since we know we will want to do matrix algebra, we can enter it directly into a numpy array.

In [18]:
assay = np.array([[0.55, 0.22,  0.15, 0.06],
                  [0.6,  0.15,  0.2,  0.02],
                  [0.4,  0.3,   0.25, 0.  ],
                  [0.4,  0.35,  0.2,  0.  ],
                  [0.5,  0.15,  0.08, 0.2 ],
                  [0.44, 0.2,   0.1,  0.22]])

Likewise, we can enter the sale prices of refined products.

In [19]:
product_price = np.array([160, 160, 120, 100])

The final piece of information we need to start solving the problem is the availability and demand constraint data.

In [20]:
availability = [80000,100000,100000,95000,75000,11000]
demand = [165000,100000,110000,80000]

**Step 2: Goal Function**

Before we get into the nitty greaty of how to use the `scipy.optimize.minimize`, let's get clever with linear algebra. The goal of this problem is to maximize profits. For a refinery, profit is difference between the money it spends to purchase crude, and the money it makes selling refined products.

For this problem, the cost, or money spent buying crude, can be calculated as the dot product of a vector (one dimensional array) of crude rate, and our `crude_cost` vector. Let's call the vector of crude rates `crude_rate`.

`total_cost = crude_cost • crude_rate`

The same trick works to calculate the revenue generated selling refined products. The dot product of a product flow rate vector, `product_rate` and the `product_prices` vector. For this to work, we will need to somehow calculate the product rate vector.

`total_revenue = product_price • product_rate`

To calculate the `product_rate` vector, we can use a different linear algebra operation, matrix multiplication. Multiplying the crude rate vector with the assay array yields a vector of the product flow rates.

`product_rate = [crude_rate][assay]`

Combining all of the above, the equation for refinery profit is:

`total_profit = product_price • ([crude_rate][assay]) - (crude_cost • crude_rate)`

Finally, we can build this expression into a python function using a few numpy operations.

In [21]:
def profit(crude_rate): # objective function
        total_profit = (np.dot(np.matmul(np.array(crude_rate), assay), product_price) \
                        - np.dot(crude_rate, crude_cost)) * -1
        return total_profit

**Step 3: Constraints**

In [ ]:
from scipy.optimize import LinearConstraint

In [22]:
avail_constraint = list(zip([0] * len(availability), availability))
demand_constraint = LinearConstraint(assay.transpose(),[0,0,0,0], demand)

**Step 4: Optimize!**

In [23]:
from scipy.optimize import minimize
guess = np.array([1,1,1,1,1,1])
results = minimize(profit, guess, constraints=demand_constraint, bounds=avail_constraint)
print(results)

     fun: -4577079.081011802
     jac: array([-16.5, -18. ,  -8. ,  -8. ,   4.5,  -9. ])
 message: 'Optimization terminated successfully.'
    nfev: 245
     nit: 29
    njev: 28
  status: 0
 success: True
       x: array([ 80000.        , 100000.        ,  65937.87850952,  74499.43243739,
            0.        ,  10966.08095736])


**Step 5: Results**

In [11]:
x = np.array(results['x'])
sold = np.matmul(x, assay)
profit = results['fun'] * -1
purchase = np.dot(x, crude_cost)
sales = np.dot(sold, product_price)

print("\n\nResults:")
print("Variable \t\t Solution \t Constraint")
print("Crude 1 (BPD):\t\t% .0f\t\t% .0f" % (x[0], availability[0]))
print("Crude 2 (BPD):\t\t% .0f\t\t% .0f" % (x[1], availability[1]))
print("Crude 3 (BPD):\t\t% .0f\t\t% .0f" % (x[2], availability[2]))
print("Crude 4 (BPD):\t\t% .0f\t\t% .0f" % (x[3], availability[3]))
print("Crude 3 (BPD):\t\t% .0f\t\t% .0f" % (x[4], availability[4]))
print("Crude 4 (BPD):\t\t% .0f\t\t% .0f\n" % (x[5], availability[5]))
print("Gasoline (BPD):\t\t% .0f\t\t% .0f" % (sold[0], demand[0]))
print("Diesel   (BPD):\t\t% .0f\t\t% .0f" % (sold[1], demand[1]))
print("Fuel Oil (BPD):\t\t% .0f\t\t% .0f" % (sold[2], demand[2]))
print("Lubes    (BPD):\t\t% .0f\t\t% .0f\n" % (sold[3], demand[2]))
print("Cost:\t\t\t$% .2f\t" % purchase)
print("Revenue:\t\t$% .2f\t" % sales)
print("Profit:\t\t\t$% .2f\n\n" % profit)



Results:
Variable 		 Solution 	 Constraint
Crude 1 (BPD):		 80000		 80000
Crude 2 (BPD):		 100000		 100000
Crude 3 (BPD):		 65938		 100000
Crude 4 (BPD):		 74499		 95000
Crude 3 (BPD):		 0		 75000
Crude 4 (BPD):		 10966		 11000

Gasoline (BPD):		 165000		 165000
Diesel   (BPD):		 80649		 100000
Fuel Oil (BPD):		 64481		 110000
Lubes    (BPD):		 9213		 110000

Cost:			$ 43385791.38	
Revenue:		$ 47962870.46	
Profit:			$ 4577079.08


